## Intro
This notebook shows how to apply Kociemba's two-phase algorithm to **Santa 2023 - The Polytope Permutation Puzzle!** competition
1. Note that two-phase algorithm does not guarantee that the produced solution is the shortest possible. Instead, it gives you a "good enough" solution in a very short time.  
2. Kociemba provides many other algorithms including optimum 'shortest' solution algorithms for cube *3x3x3*

Have fun! Many thanks to the hosts and authors mentioned below:

# Refferences/credits:
* https://pypi.org/project/kociemba/
* http://kociemba.org/
* https://github.com/hkociemba/RubiksCube-TwophaseSolver/blob/master/enums.py (UBL)
* https://www.kaggle.com/code/ryanholbrook/getting-started-with-santa-2023
* https://www.kaggle.com/code/paulorzp/magic-cube-utilities (moves)
* https://www.kaggle.com/code/metric/santa-2023-metric (scoring)
* https://www.kaggle.com/code/jazivxt/wayfinder (scoring)
* https://www.kaggle.com/code/dinhttrandrise/bidirectional-brute-force-w-wildcards/ (current best submission.csv)

In [ ]:
!pip install kociemba > NUL


  error: subprocess-exited-with-error
  
  × Building wheel for kociemba (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [80 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-310
      creating build\lib.win-amd64-cpython-310\kociemba
      copying kociemba\build_ckociemba.py -> build\lib.win-amd64-cpython-310\kociemba
      copying kociemba\command_line.py -> build\lib.win-amd64-cpython-310\kociemba
      copying kociemba\__init__.py -> build\lib.win-amd64-cpython-310\kociemba
      creating build\lib.win-amd64-cpython-310\kociemba\cprunetables
      copying kociemba\cprunetables\flipMove -> build\lib.win-amd64-cpython-310\kociemba\cprunetables
      copying kociemba\cprunetables\FRtoBR_Move -> build\lib.win-amd64-cpython-310\kociemba\cprunetables
      copying kociemba\cprunetables\MergeURtoULandUBtoDF -> build\lib.win-amd64-cpython-310\kociemba\cprunetables
      cop

In [ ]:
! pip install kociemba > /dev/null
import kociemba

The system cannot find the path specified.


ModuleNotFoundError: No module named 'kociemba'

## Initial packages/data load
* https://www.kaggle.com/code/metric/santa-2023-metric
* https://www.kaggle.com/code/jazivxt/wayfinder
* https://www.kaggle.com/code/dinhttrandrise/bidirectional-brute-force-w-wildcards/

In [ ]:
import pandas as pd
import numpy as np
from sympy.combinatorics import Permutation

p = 'C:/Users/enesm/OneDrive/Masaüstü/KAGGLE/Santa 2023 - The Polytope Permutation Puzzle/'
path = pd.read_csv('puzzles.csv')
info = pd.read_csv('puzzle_info.csv')
sub = pd.read_csv('submission.csv')

info['allowed_moves_count'] = info['allowed_moves'].map(lambda x: {k: Permutation(v) for k, v in eval(x).items()})
paths = pd.merge(path, info, how='left', on='puzzle_type')
paths = pd.merge(paths, sub, how='left', on='id')

## Convert state to UBL 
Note that competition data is different than standard notation. Function `state2ubl()` performs two steps: mapping centre to UBL and then converts string into standard notation. Based on:
* https://github.com/hkociemba/RubiksCube-TwophaseSolver/blob/master/enums.py (UBL)
* https://www.kaggle.com/code/ryanholbrook/getting-started-with-santa-2023

In [ ]:
U = ['U', 'F', 'R', 'B', 'L', 'D']

def state2ubl(state):

    dct = {}
    for u in range(len(U)):
        dct[state.split(';')[4+u*9]] = U[u]

    s = ''.join([dct[f] for f in state.split(';')])
    
    return s[:9] + s[18:27] + s[9:18] + s[45:] + s[36:45] + s[27:36]

## Converting moves
* https://www.kaggle.com/code/paulorzp/magic-cube-utilities

In [ ]:
moves = eval(info.loc[info['puzzle_type'] == 'cube_3/3/3']['allowed_moves'].values[0])
for move in list(moves):
    moves['-'+move] = np.argsort(moves[move]).tolist()

M = {}
M["U"] = '-d2'
M["R"] = "r0"
M["B"] = "-f2"
M["F"] = "f0"
M["L"] = "-r2"
M["D"] = "d0"
for m in list(M):
    M[m+"2"] = M[m] + '.' + M[m]
    if "-" in M[m]:
        M[m+"'"] = M[m].replace("-","")
    else:
        M[m+"'"] = "-"+M[m]

In [ ]:
for i in paths.loc[paths['puzzle_type'] == 'cube_3/3/3'].iterrows():
    try:
        id = i[1]['id']
        cur_state = i[1]['initial_state']
        
        sol = kociemba.solve(state2ubl(cur_state))

        new_state = cur_state
        mmoves = '.'.join([M[m] for m in sol.split(' ')])

        for move in mmoves.split('.'):
            new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))
        I = ['r0.r1.r2','d0.d1.d2','f0.f1.f2']
        for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
            temp_state = new_state
            if len(init_moves) > 0:
                for move in init_moves.split('.'):
                    temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))

            if temp_state == i[1]['solution_state']:
                print(f'solved id: {id}')
                if len(init_moves) > 0:
                    mmoves += '.' + init_moves
                if len(paths.iloc[id,7].split('.')) > len(mmoves.split('.')):
                    print(f"improved: new length {len(mmoves.split('.'))} vs current length {len(paths.iloc[id,7].split('.'))}")
                    paths.iloc[id,7] = mmoves
                break
    except:
        pass

## Score
* https://www.kaggle.com/code/metric/santa-2023-metric

In [7]:
class ParticipantVisibleError(Exception):
    pass

def score(sol) -> float:
    total_num_moves = 0
    for i in range(len(sol)):
        puzzle_id=sol['id'][i]
        moves = sol.moves[i].split('.')
        allowed_moves=sol.allowed_moves_count[i]
        state = sol.initial_state[i].split(';')
        solution_state=sol.solution_state[i].split(';')
        num_wildcards=sol.num_wildcards[i]
        for m in moves:
            power = 1
            if m[0] == "-":
                m = m[1:]
                power = -1
            try:
                p = allowed_moves[m]
            except KeyError:
                raise ParticipantVisibleError(f"{m} is not an allowed move for {puzzle_id}.")
            state = (p ** power)(state)
        num_wrong_facelets = sum(not(s == t) for s, t in zip(solution_state, state))
        if num_wrong_facelets > num_wildcards:
            print(puzzle_id, num_wrong_facelets, puzzle.num_wildcards)
            raise ParticipantVisibleError(f"Submitted moves do not solve {puzzle_id}.")
        total_num_moves += len(moves)
    return total_num_moves

In [8]:
%%time
score(paths)

KeyboardInterrupt: 

# Submission

In [ ]:
paths[['id','moves']].to_csv('submission.csv', index=False)
! head -n 3 submission.csv

id,moves
0,r1.-f1
1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
